In [6]:
import pandas as pd 
import json 
import os, sys
import re
from collections import defaultdict, Counter 
from statistics import mean, stdev

In [2]:
def get_dataset_name(line):
    line_list = line.strip().split(", ") 
    for item in line_list:
        item_split = item.split(":")
        if 'data_dir' in item_split[0]:
            dataset_name = "-".join(item_split[1].split("/")[1:-1])
    return dataset_name


In [28]:

def extract_multi_runs_results(path):
    # '''key is config, values are the multiple runs of p@10 and mrr'''
    dataset_to_res_def_dap = defaultdict(list)
    dataset_to_res_lsp_dap = defaultdict(list)
    keys = ['mask_type', 'p@1', 'p@5', 'p@10', 'mrr', 'mask_type_id'] 
    with open(path, 'r') as fin:
        lines = fin.readlines()
        for i, line in enumerate(lines):

            if '--- config --' in line and lines[i+1].startswith("{"): 
                config = eval(lines[i+1])
                dataset= config['dataset']
                #print(config) 
                #print("dataset ", dataset) 

            if " mask_type   	   p@1 	   p@5 	   p@10 	   mrr 	   mask_type_id " in line:
                def_sap = [item.strip() for item in lines[i+2].split("\t")][1:]
                def_dap = [item.strip() for item in lines[i+4].split("\t")][1:]
                lsp_dap = [item.strip() for item in lines[i+5].split("\t")][1:]
                
                def_sap_res = dict(zip(keys, def_sap))
                def_dap_res = dict(zip(keys, def_dap))
                lsp_dap_res = dict(zip(keys, lsp_dap))
                
                def_sap_res['dataset'] = dataset 
                def_dap_res['dataset'] = dataset 
                lsp_dap_res['dataset'] = dataset 
                #print(lsp_dap_res)
    
            if ".csv" in line: 
                res_path = line.split(" ")[1].strip()
                dataset_to_res_def_dap[res_path+dataset].append(def_dap_res)
                dataset_to_res_lsp_dap[res_path+dataset].append(lsp_dap_res)
                
    return dataset_to_res_def_dap, dataset_to_res_lsp_dap

def get_average(dataset_to_res):
    for k,v in dataset_to_res.items():
        mean_p_at_10 = round(mean([eval(item['p@10']) for item in v]), 1)
        mean_mrr = round( mean([eval(item['mrr']) for item in v]), 1)
        
        stdev_p_at_10 = round(stdev([eval(item['p@10']) for item in v]), 1)
        stdev_mrr = round(stdev([eval(item['mrr']) for item in v]), 1)
        print (v[0]['dataset'], f'mrr and p_at_10: {mean_mrr} &  {mean_p_at_10} & {stdev_mrr} {stdev_p_at_10}')

# path = '../../log/230313_wordnet_alone_2K.log'
# path = '../../log/230313_wordnet_six_datasets.log'
# path = '../../log/230313_with_swow_2k_anchors_multiple_runs.log'
# path = '../../log/230313_swow_share.log'
# path = '../../log/230314_wordnet_share_2K.log'
path = '../../log/230314_only_swow_2k_anchors_multiple_runs.log'
dataset_to_res_def_dap, dataset_to_res_lsp_dap = extract_multi_runs_results(path)
get_average(dataset_to_res_def_dap) 
print('lsp_dap')
get_average(dataset_to_res_lsp_dap) 

# print(json.dumps(dataset_to_res, indent=4))


ALLSWOW mrr and p_at_10: 24.0 &  49.3 & 0.1 0.1
ALLSWOW mrr and p_at_10: 24.2 &  49 & 0.0 0.0
lsp_dap
ALLSWOW mrr and p_at_10: 21.6 &  45.3 & 0.0 0.0
ALLSWOW mrr and p_at_10: 22.5 &  45.7 & 0.0 0.0


In [27]:
path = '../../log/bert-large-uncased/ALL/swow_rw/df_all_use_global_dap_True_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_anchor_source_WordNet_wns_False_wnf_True_swow_score_source_None.ALLSWOW.csv'

df = pd.read_csv(path)
# df.columns
df['subj_anchors_wordnet_all_num'] = df['subj_anchors_wordnet_all'].apply(lambda x: len(eval(x)))
df['subj_anchors_wordnet_all_num'].value_counts(normalize=True)


10    0.366762
1     0.173352
2     0.124164
9     0.086437
3     0.081662
4     0.065425
5     0.053486
6     0.019580
7     0.014804
8     0.014327
Name: subj_anchors_wordnet_all_num, dtype: float64

In [18]:
path = '../../log/bert-large-uncased/ALL/swow_rw/df_all_use_global_dap_True_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_anchor_source_SWOW_wns_False_wnf_False_swow_score_source_OnlySWOWSimilar.ALLSWOW.csv'

df = pd.read_csv(path)
df.columns
# df[['sub_label', 'obj_label', 'subj_anchors_swow_sg']]

Index(['Unnamed: 0', 'sub_label', 'obj_label', 'relation', 'obj_in_BERT',
       'masked_sentences', 'uuid', 'sub_position', 'sub_label_sg',
       'sub_label_pl', 'def_sap', 'def_dap', 'lsp_sap', 'lsp_dap',
       'anchor_lsp_sap', 'sub_label_sgpl', 'obj_mask_sentence',
       'obj_mask_sentence_score', 'obj_mask_def_sap', 'obj_mask_def_sap_score',
       'obj_mask_lsp_sap', 'obj_mask_lsp_sap_score', 'subj_anchors',
       'subj_anchors_score', 'subj_anchors_sg', 'subj_anchors_pl',
       'subj_anchors_all', 'subj_anchors_combined', 'anchor_num',
       'def_dap_with_subj_anchor', 'lsp_dap_with_subj_anchor',
       'obj_mask_def_dap', 'obj_mask_def_dap_score', 'obj_mask_lsp_dap',
       'obj_mask_lsp_dap_score', 'sub_sister', 'obj_label_sg', 'p1_sentence',
       'p5_sentence', 'p10_sentence', 'p1_def_sap', 'p5_def_sap',
       'p10_def_sap', 'p1_lsp_sap', 'p5_lsp_sap', 'p10_lsp_sap', 'p1_def_dap',
       'p5_def_dap', 'p10_def_dap', 'p1_lsp_dap', 'p5_lsp_dap', 'p10_lsp_dap',
       '